In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1181251,2021-04-14,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1181252,2021-04-15,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1181253,2021-04-16,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1181254,2021-04-17,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
31651,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
31653,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
31655,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
31657,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
31659,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
727889,2021-04-14,Arkansas,Arkansas,5001,2085,36.00,5000,Arkansas,AR,Arkansas,State,3017804
727891,2021-04-15,Arkansas,Arkansas,5001,2086,36.00,5000,Arkansas,AR,Arkansas,State,3017804
727893,2021-04-16,Arkansas,Arkansas,5001,2086,36.00,5000,Arkansas,AR,Arkansas,State,3017804
727895,2021-04-17,Arkansas,Arkansas,5001,2086,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1181251,2021-04-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1181252,2021-04-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1181253,2021-04-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1181254,2021-04-17,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-04-18') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
453,2021-04-18,Snohomish,Washington,34236,564.00,53061,WA,County,822083,4164.54,68.61
904,2021-04-18,Cook,Illinois,521694,10379.00,17031,IL,County,5150233,10129.52,201.52
1354,2021-04-18,Orange,California,268962,4896.00,6059,CA,County,3175692,8469.40,154.17
1803,2021-04-18,Maricopa,Arizona,531156,9767.00,4013,AZ,County,4485414,11841.85,217.75
2252,2021-04-18,Los Angeles,California,1229659,23653.00,6037,CA,County,10039107,12248.69,235.61
...,...,...,...,...,...,...,...,...,...,...,...
1180627,2021-04-18,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08
1180815,2021-04-18,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1180972,2021-04-18,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1181125,2021-04-18,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
453,2021-04-18,Snohomish,Washington,34236,564.00,53061,WA,County,822083,4164.54,68.61,68.61,4164.54
904,2021-04-18,Cook,Illinois,521694,10379.00,17031,IL,County,5150233,10129.52,201.52,201.52,10129.52
1354,2021-04-18,Orange,California,268962,4896.00,6059,CA,County,3175692,8469.40,154.17,154.17,8469.40
1803,2021-04-18,Maricopa,Arizona,531156,9767.00,4013,AZ,County,4485414,11841.85,217.75,217.75,11841.85
2252,2021-04-18,Los Angeles,California,1229659,23653.00,6037,CA,County,10039107,12248.69,235.61,235.61,12248.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1180627,2021-04-18,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08,75.08,1876.88
1180815,2021-04-18,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1180972,2021-04-18,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1181125,2021-04-18,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
761090,2021-04-18,Hale,Alabama,2186,75.00,1065,AL,County,14651,14920.48,511.91,511.91,14920.48,1
828385,2021-04-18,Clarke,Alabama,3473,60.00,1025,AL,County,23622,14702.40,254.00,254.00,14702.40,2
595581,2021-04-18,Lowndes,Alabama,1373,53.00,1085,AL,County,9726,14116.80,544.93,544.93,14116.80,3
443196,2021-04-18,Franklin,Alabama,4213,82.00,1059,AL,County,31362,13433.45,261.46,261.46,13433.45,4
528117,2021-04-18,Etowah,Alabama,13703,348.00,1055,AL,County,102268,13399.11,340.28,340.28,13399.11,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120416,2021-04-18,Platte,Wyoming,617,11.00,56031,WY,County,8393,7351.36,131.06,131.06,7351.36,19
726352,2021-04-18,Converse,Wyoming,991,17.00,56009,WY,County,13822,7169.73,122.99,122.99,7169.73,20
902877,2021-04-18,Lincoln,Wyoming,1386,12.00,56023,WY,County,19830,6989.41,60.51,60.51,6989.41,21
1004753,2021-04-18,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
595581,2021-04-18,Lowndes,Alabama,1373,53.00,1085,AL,County,9726,14116.80,544.93,544.93,14116.80,3,1
761090,2021-04-18,Hale,Alabama,2186,75.00,1065,AL,County,14651,14920.48,511.91,511.91,14920.48,1,2
253402,2021-04-18,Walker,Alabama,7149,276.00,1127,AL,County,63521,11254.55,434.50,434.50,11254.55,28,3
642258,2021-04-18,Greene,Alabama,914,34.00,1063,AL,County,8111,11268.65,419.18,419.18,11268.65,27,4
526557,2021-04-18,Clay,Alabama,1502,55.00,1027,AL,County,13235,11348.70,415.56,415.56,11348.70,24,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640705,2021-04-18,Hot Springs,Wyoming,364,3.00,56017,WY,County,4413,8248.36,67.98,67.98,8248.36,17,19
902877,2021-04-18,Lincoln,Wyoming,1386,12.00,56023,WY,County,19830,6989.41,60.51,60.51,6989.41,21,20
880398,2021-04-18,Uinta,Wyoming,2155,12.00,56041,WY,County,20226,10654.60,59.33,59.33,10654.60,4,21
252214,2021-04-18,Teton,Wyoming,3694,9.00,56039,WY,County,23464,15743.27,38.36,38.36,15743.27,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4164.54,19,23
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4164.54,19,23
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4164.54,19,23
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4164.54,19,23
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.61,4164.54,19,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1178479,2021-04-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1178480,2021-04-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1178481,2021-04-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1178482,2021-04-17,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
757934,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14920.48,2,1,1.00
757935,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14920.48,2,1,0.00
757936,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14920.48,2,1,0.00
757937,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14920.48,2,1,0.00
757938,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,511.91,14920.48,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917961,2021-04-14,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5643.46,6,23,0.00
917962,2021-04-15,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5643.46,6,23,0.00
917963,2021-04-16,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5643.46,6,23,0.00
917964,2021-04-17,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5643.46,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
593195,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14116.80,1,3,1.00,0.00
593196,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14116.80,1,3,0.00,0.00
593197,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14116.80,1,3,0.00,0.00
593198,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14116.80,1,3,0.00,0.00
593199,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14116.80,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591245,2021-04-14,Albany,Wyoming,4096,11.00,56001,WY,County,38880,10534.98,28.29,28.29,10594.14,23,5,3.00,0.00
591246,2021-04-15,Albany,Wyoming,4105,11.00,56001,WY,County,38880,10558.13,28.29,28.29,10594.14,23,5,9.00,0.00
591247,2021-04-16,Albany,Wyoming,4119,11.00,56001,WY,County,38880,10594.14,28.29,28.29,10594.14,23,5,14.00,0.00
591248,2021-04-17,Albany,Wyoming,4119,11.00,56001,WY,County,38880,10594.14,28.29,28.29,10594.14,23,5,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-04-18') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
302474,2021-04-18,Imperial,California,27987,718.00,6025,CA,County,181215,15444.09,396.21,396.21,15444.09,1,2,25.00,0.00
2252,2021-04-18,Los Angeles,California,1229659,23653.00,6037,CA,County,10039107,12248.69,235.61,235.61,12248.69,2,5,433.00,26.00
150660,2021-04-18,San Bernardino,California,294779,4555.00,6071,CA,County,2180085,13521.45,208.94,208.94,13521.45,3,4,202.00,5.00
529629,2021-04-18,Inyo,California,1392,36.00,6027,CA,County,18039,7716.61,199.57,199.57,7716.61,4,25,0.00,0.00
67393,2021-04-18,Stanislaus,California,60383,1028.00,6099,CA,County,550660,10965.57,186.69,186.69,10965.57,5,9,0.00,0.00
31242,2021-04-18,Riverside,California,297215,4525.00,6065,CA,County,2470546,12030.34,183.16,183.16,12030.34,6,6,0.00,0.00
83550,2021-04-18,Tulare,California,49383,829.00,6107,CA,County,466195,10592.78,177.82,177.82,10592.78,7,10,0.00,-1.00
53625,2021-04-18,San Joaquin,California,71431,1334.00,6077,CA,County,762148,9372.33,175.03,175.03,9372.33,8,19,0.00,0.00
30426,2021-04-18,Fresno,California,100562,1646.00,6019,CA,County,999101,10065.25,164.75,164.75,10065.25,9,14,62.00,3.00
479989,2021-04-18,Merced,California,31196,452.00,6047,CA,County,277680,11234.51,162.78,162.78,11234.51,10,8,0.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1119967,2021-04-18,Lassen,California,5669,1.00,6035,CA,County,30573,18542.50,3.27,3.27,18542.50,56,1,0.00,0.00
302474,2021-04-18,Imperial,California,27987,718.00,6025,CA,County,181215,15444.09,396.21,396.21,15444.09,1,2,25.00,0.00
642200,2021-04-18,Kings,California,22774,245.00,6031,CA,County,152940,14890.81,160.19,160.19,14890.81,11,3,8.00,0.00
150660,2021-04-18,San Bernardino,California,294779,4555.00,6071,CA,County,2180085,13521.45,208.94,208.94,13521.45,3,4,202.00,5.00
2252,2021-04-18,Los Angeles,California,1229659,23653.00,6037,CA,County,10039107,12248.69,235.61,235.61,12248.69,2,5,433.00,26.00
31242,2021-04-18,Riverside,California,297215,4525.00,6065,CA,County,2470546,12030.34,183.16,183.16,12030.34,6,6,0.00,0.00
189405,2021-04-18,Kern,California,107859,1322.00,6029,CA,County,900202,11981.64,146.86,146.86,11981.64,14,7,34.00,0.00
479989,2021-04-18,Merced,California,31196,452.00,6047,CA,County,277680,11234.51,162.78,162.78,11234.51,10,8,0.00,0.00
67393,2021-04-18,Stanislaus,California,60383,1028.00,6099,CA,County,550660,10965.57,186.69,186.69,10965.57,5,9,0.00,0.00
83550,2021-04-18,Tulare,California,49383,829.00,6107,CA,County,466195,10592.78,177.82,177.82,10592.78,7,10,0.00,-1.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
302474,2021-04-18,Imperial,California,27987,718.00,6025,CA,County,181215,15444.09,396.21,396.21,15444.09,1,2,25.00,0.00
2252,2021-04-18,Los Angeles,California,1229659,23653.00,6037,CA,County,10039107,12248.69,235.61,235.61,12248.69,2,5,433.00,26.00
150660,2021-04-18,San Bernardino,California,294779,4555.00,6071,CA,County,2180085,13521.45,208.94,208.94,13521.45,3,4,202.00,5.00
529629,2021-04-18,Inyo,California,1392,36.00,6027,CA,County,18039,7716.61,199.57,199.57,7716.61,4,25,0.00,0.00
67393,2021-04-18,Stanislaus,California,60383,1028.00,6099,CA,County,550660,10965.57,186.69,186.69,10965.57,5,9,0.00,0.00
31242,2021-04-18,Riverside,California,297215,4525.00,6065,CA,County,2470546,12030.34,183.16,183.16,12030.34,6,6,0.00,0.00
83550,2021-04-18,Tulare,California,49383,829.00,6107,CA,County,466195,10592.78,177.82,177.82,10592.78,7,10,0.00,-1.00
53625,2021-04-18,San Joaquin,California,71431,1334.00,6077,CA,County,762148,9372.33,175.03,175.03,9372.33,8,19,0.00,0.00
30426,2021-04-18,Fresno,California,100562,1646.00,6019,CA,County,999101,10065.25,164.75,164.75,10065.25,9,14,62.00,3.00
479989,2021-04-18,Merced,California,31196,452.00,6047,CA,County,277680,11234.51,162.78,162.78,11234.51,10,8,0.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,396.21,15444.09,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,396.21,15444.09,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,396.21,15444.09,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,396.21,15444.09,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,396.21,15444.09,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3962,04/14/21,Lassen,5666,1.00,18532.69,3.27,3.27,18542.50,56,1,0.00,0.00
3963,04/15/21,Lassen,5666,1.00,18532.69,3.27,3.27,18542.50,56,1,0.00,0.00
3964,04/16/21,Lassen,5669,1.00,18542.50,3.27,3.27,18542.50,56,1,3.00,0.00
3965,04/17/21,Lassen,5669,1.00,18542.50,3.27,3.27,18542.50,56,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,396.21,15444.09,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,396.21,15444.09,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,396.21,15444.09,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,396.21,15444.09,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,396.21,15444.09,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4048,04/14/21,Merced,31128,450.00,11210.03,162.06,162.78,11234.51,10,8,23.00,1.00
4049,04/15/21,Merced,31151,452.00,11218.31,162.78,162.78,11234.51,10,8,23.00,2.00
4050,04/16/21,Merced,31196,452.00,11234.51,162.78,162.78,11234.51,10,8,45.00,0.00
4051,04/17/21,Merced,31196,452.00,11234.51,162.78,162.78,11234.51,10,8,0.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)